In [178]:
import pandas as pd
import numpy as np

In [179]:
df = pd.read_csv("inputs/month_sheet.csv")
rate = pd.read_csv("inputs/salary_sheet.csv")

In [180]:
df = df[['Client','Assign to','Time']]
df = df.rename(columns={'Assign to':'Employee'})

In [181]:
customer = pd.read_csv("inputs/client_sheet.csv")

In [182]:
df["Time"].isna().sum()

229

In [183]:
df = df.merge(rate)
df = df.merge(customer)

In [184]:
df["Time"].isna().sum()

174

In [185]:

df.loc[df['Time'] == 'all day', 'Time'] = '480'

df.loc[df['Time'] == 'done earlier', 'Time'] = np.nan


In [186]:
df = df.dropna()


In [187]:
df.loc[:, 'Time'] = df.loc[:, 'Time'].astype(int)
df.loc[:, 'salary'] = df.loc[:, 'salary'].astype(int)


monthly salary = 22 days = 22\*24 hrs = 22\*8\*60 mins


In [188]:
df.loc[:,"minute_rate"] = df["salary"]/(22*8*60)

In [189]:
df.loc[:,'spend'] = df['Time']*df['minute_rate'] 

In [190]:
pd.set_option('future.no_silent_downcasting', True)
df_clientwise = df.pivot_table(index='Client',columns='Employee',values=['spend']).fillna(0)

In [191]:
df_clientwise['total_spend'] = df_clientwise.sum(axis=1).apply(int)

## Flatenning of table

In [192]:
df_profit = df_clientwise.reset_index()

In [193]:
df_profit.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_profit.columns]

In [194]:
df_profit = df_profit.merge(customer)

In [195]:
df_profit = df_profit[['Client','total_spend','Payment']]

In [196]:
df_profit['profit'] = df_profit['Payment'] - df_profit['total_spend'] 

In [197]:
df_empwise = df.pivot_table(index='Employee',columns='Client',values=['Time']).fillna(0)
df_empwise['Total_time'] = df_empwise.sum(axis=1).apply(int)

In [198]:
df_empwise = df_empwise.reset_index()
df_empwise.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_empwise.columns]

In [199]:
df_timewise = df.pivot_table(index='Client',columns='Employee',values=['Time']).fillna(0)
df_timewise['Total_time_(mins)'] = df_timewise.sum(axis=1).apply(float)
df_timewise['Total_time_(hrs)'] = df_timewise.loc[:,"Total_time_(mins)"]/60
df_timewise = df_timewise.reset_index()
df_timewise.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_timewise.columns]

### new data
- total time spent on each project 
- total time spent by each employee on each project 
- total time spent by emp on projects in a month duration
- total profit from each project